In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# !pip install playwright > /dev/null
# !pip install  lxml

# If this is your first time using playwright, you'll have to install a browser executable.
# Running `playwright install` by default installs a chromium browser executable.
# playwright install

In [3]:
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import run_async

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio
nest_asyncio.apply()

In [10]:
from playwright.async_api import async_playwright

playwright = async_playwright()
device = {
    "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.53 Safari/537.36",
    "screen": {
      "width": 1920,
      "height": 1080
    },
    "viewport": {
      "width": 1280,
      "height": 720
    },
    "device_scale_factor": 1,
    "is_mobile": False,
    "has_touch": False,
    "default_browser_type": "chromium"
}
browser = run_async(playwright.start())
browser = run_async(browser.chromium.launch(headless=True))
context = await browser.new_context(**device)

toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=browser)
tools = toolkit.get_tools()

tools_by_name = {tool.name: tool for tool in tools}
navigate_tool = tools_by_name["navigate_browser"]
get_elements_tool = tools_by_name["get_elements"]

url = "https://www.ftvnews.com.tw/news/detail/2023505W0297"
await navigate_tool.arun({"url": url})

'Navigating to https://www.ftvnews.com.tw/news/detail/2023505W0297 returned status code 200'

In [24]:
text = await get_elements_tool.arun({"selector": "article"})

import json

h = json.loads(text)
article = h[0]["innerText"]
print(article)

趙怡翔說：國民黨頭痛的是，該如何把郭台銘「輕輕地放下」。

論壇中心／林靜芬報導

國民黨2024總統人選尚未底定，新北市長侯友宜與鴻海創辦人郭台銘實力相當，但如今傳出國民黨已內定侯出馬參選。對此，民進黨台北市議員趙怡翔在《全國第一勇》節目中表示，郭台銘現在已經亂了、急了，「因為他知道他快被做掉了！」而國民黨接下來要思考的是，「如何把郭台銘輕輕地放下，一旦放得太快、破碎了，對不起，他又再次跟你國民黨翻臉」。










更多新聞： 快新聞／出獄畫面曝！　趙玉柱獲假釋手比讚：非常高興

郭台銘與侯友宜的國民黨總統參選徵召之爭，越趨白熱化。《全國第一勇》來賓趙怡翔指出，郭台銘最近拋出的許多議題，被批評欠缺周全的思考，包括「我要用AI去處理詐騙、我要用機器人去維護台灣的戰場」等等，為什麼這樣說？「因為他急了，他知道在整個過程當中已經都被內定了」。







趙怡翔：侯、郭像打電動兩兄弟， 只有一個人真玩。（圖／民視新聞）




趙怡翔分析指出，郭台銘最近的處境，讓他想到網路上一張迷因哏圖，「就是兩個兄弟在打電動，兩個都打得很認真，但只有哥（侯友宜）遙控器有插進電玩裡面，弟弟（郭台銘）的是完全沒有插進去。弟弟就是在打假球，自high而已」。

趙怡翔進一步表示：「所以我就說，只有侯友宜真的在玩，郭台銘以為他在玩，但他連插頭都沒插進去，不知道在玩什麼，但重點是，根本從頭到尾就沒有他的份，只不過還是有畫面」、「因為侯友宜也在按，就是哥哥也在按，弟弟按得很開心，以為是他在跳，但其實是哥哥在跳。」

但郭董可以任憑國民黨愚弄嗎？趙怡翔認為，現在國民黨如果要說服大家，這個是一個公正的選舉，就應該把民調的基準、時間點拿出來，「你把到時候考慮的，不同因素跟百分比全部都拿出來，說對不起，客觀而言就是侯友宜比較強，我覺得這會買單的」，但是今天如果你是黑箱作業，說不出來任何的依據，最後就推侯友宜的話，國民黨會有麻煩。
「為什麼？你把他放得太快、 破碎了，他又再次跟你國民黨翻臉，甚至於去加入第三黨，所以國民黨現在要思考的就是，要如何把郭台銘輕輕地放下。」

更多新聞： 賴清德新北民調超車侯友宜6％　立委分析「2關鍵」侯失去優勢


In [25]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.callbacks import get_openai_callback

# llm = ChatOpenAI(model_name="gpt-4-0314", temperature=0)
llm = ChatOpenAI(temperature=0)

# prompt = f"""
# navigate to {url} and article of the news.
# """
prompt = f"""
將以下文章總結成最多100字：

{article}
"""

with get_openai_callback() as cb:
    result = llm([HumanMessage(content=prompt)])
    # result = await agent_chain.arun(prompt)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
    print(result)

Total Tokens: 1497
Prompt Tokens: 1322
Completion Tokens: 175
Total Cost (USD): $0.002994
content='國民黨內定侯友宜參選2024總統，讓郭台銘感到亂、急，但國民黨頭痛的是如何輕輕地放下他，避免再次翻臉。趙怡翔指出，郭台銘像打電動的弟弟，只是自high而已，而國民黨應該公正選舉，否則會有麻煩。國民黨要思考如何放下郭台銘，避免他加入第三黨。' additional_kwargs={} example=False


In [23]:
import json

h = json.loads(text)
h[0]["innerText"]

'趙怡翔說：國民黨頭痛的是，該如何把郭台銘「輕輕地放下」。\n\n論壇中心／林靜芬報導\n\n國民黨2024總統人選尚未底定，新北市長侯友宜與鴻海創辦人郭台銘實力相當，但如今傳出國民黨已內定侯出馬參選。對此，民進黨台北市議員趙怡翔在《全國第一勇》節目中表示，郭台銘現在已經亂了、急了，「因為他知道他快被做掉了！」而國民黨接下來要思考的是，「如何把郭台銘輕輕地放下，一旦放得太快、破碎了，對不起，他又再次跟你國民黨翻臉」。\n\n\n\n\n\n\n\n\n\n\n更多新聞： 快新聞／出獄畫面曝！\u3000趙玉柱獲假釋手比讚：非常高興\n\n郭台銘與侯友宜的國民黨總統參選徵召之爭，越趨白熱化。《全國第一勇》來賓趙怡翔指出，郭台銘最近拋出的許多議題，被批評欠缺周全的思考，包括「我要用AI去處理詐騙、我要用機器人去維護台灣的戰場」等等，為什麼這樣說？「因為他急了，他知道在整個過程當中已經都被內定了」。\n\n\n\n\n\n\n\n趙怡翔：侯、郭像打電動兩兄弟， 只有一個人真玩。（圖／民視新聞）\n\n\n\n\n趙怡翔分析指出，郭台銘最近的處境，讓他想到網路上一張迷因哏圖，「就是兩個兄弟在打電動，兩個都打得很認真，但只有哥（侯友宜）遙控器有插進電玩裡面，弟弟（郭台銘）的是完全沒有插進去。弟弟就是在打假球，自high而已」。\n\n趙怡翔進一步表示：「所以我就說，只有侯友宜真的在玩，郭台銘以為他在玩，但他連插頭都沒插進去，不知道在玩什麼，但重點是，根本從頭到尾就沒有他的份，只不過還是有畫面」、「因為侯友宜也在按，就是哥哥也在按，弟弟按得很開心，以為是他在跳，但其實是哥哥在跳。」\n\n但郭董可以任憑國民黨愚弄嗎？趙怡翔認為，現在國民黨如果要說服大家，這個是一個公正的選舉，就應該把民調的基準、時間點拿出來，「你把到時候考慮的，不同因素跟百分比全部都拿出來，說對不起，客觀而言就是侯友宜比較強，我覺得這會買單的」，但是今天如果你是黑箱作業，說不出來任何的依據，最後就推侯友宜的話，國民黨會有麻煩。\n「為什麼？你把他放得太快、 破碎了，他又再次跟你國民黨翻臉，甚至於去加入第三黨，所以國民黨現在要思考的就是，要如何把郭台銘輕輕地放下。」\n\n更多新聞： 賴清德新北民調超車侯友宜6％\u3000立委分析「2關鍵」侯失去優勢'